<a href="https://colab.research.google.com/github/ffreirea1988/usm/blob/pega/Carga_gastos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#Nombre del archivo (se modifica mes a mes)
nmb_input = 'Cuentas TI - 092023 - Septiembre 2023.xlsx'

In [3]:
#Mes de cierre
mes = 9

In [4]:
df_2701 = pd.read_excel(f'/content/{nmb_input}', sheet_name='31112701')
df_2702 = pd.read_excel(f'/content/{nmb_input}', sheet_name='31112702')
df_2703 = pd.read_excel(f'/content/{nmb_input}', sheet_name='31112703')
df_2704 = pd.read_excel(f'/content/{nmb_input}', sheet_name='31112704')
df_2705 = pd.read_excel(f'/content/{nmb_input}', sheet_name='31112705')
df_2710 = pd.read_excel(f'/content/{nmb_input}', sheet_name='31112710')
df_2711 = pd.read_excel(f'/content/{nmb_input}', sheet_name='31112711')
df_2712 = pd.read_excel(f'/content/{nmb_input}', sheet_name='31112712')

In [5]:
#Eliminar la última fila de cada tabla (n=1)
n=1
df_2701.drop(df_2701.tail(n).index, inplace = True)
df_2702.drop(df_2702.tail(n).index, inplace = True)
df_2703.drop(df_2703.tail(n).index, inplace = True)
df_2704.drop(df_2704.tail(n).index, inplace = True)
df_2705.drop(df_2705.tail(n).index, inplace = True)
df_2710.drop(df_2710.tail(n).index, inplace = True)
df_2711.drop(df_2711.tail(n).index, inplace = True)
df_2712.drop(df_2712.tail(n).index, inplace = True)


In [6]:
#Concatenar archivos en un solo dataframe con todas las cuentas
#axis=1 unir por filas
#axis=0 unir por columnas
df = pd.concat([df_2701, df_2702, df_2703, df_2704, df_2705, df_2710, df_2711, df_2712], axis=0)

In [7]:
#Eliminar mes no cerrado:
df.drop(df[(df['Período contable'] > mes)].index, inplace=True)

In [8]:
#Revisar columnas
df.columns

Index(['Período contable', 'Texto cab.documento', 'Texto', 'Cuenta',
       'Importe en moneda local', 'Moneda local', 'Centro de coste',
       'Referencia', 'Material', 'Nombre del usuario', 'Nº documento',
       'Fecha de documento', 'Fe.contabilización', 'Importe en ML3',
       'Clase de documento', 'Orden', 'Sociedad', 'Documento compras', 'Grafo',
       'Elemento PEP', 'Importe libro mayor', 'Moneda libro mayor',
       'Tp.cambio efectivo', 'Posición', 'Ejercicio / mes',
       'Clave contabiliz.'],
      dtype='object')

In [9]:
df['Cuenta'].unique()

array([31112701., 31112702., 31112703., 31112704., 31112705., 31112710.,
       31112711., 31112712.])

In [10]:
df['Período contable'].unique()

array([1., 2., 3., 4., 5., 6., 7., 8., 9.])

In [11]:
print(df.shape)
df.isnull().sum()
#df.dtypes

(24267, 26)


Período contable               0
Texto cab.documento           13
Texto                        940
Cuenta                         0
Importe en moneda local        0
Moneda local                   0
Centro de coste               23
Referencia                  9129
Material                   24266
Nombre del usuario             0
Nº documento                   0
Fecha de documento             0
Fe.contabilización             0
Importe en ML3                 0
Clase de documento             0
Orden                      24267
Sociedad                       0
Documento compras          20518
Grafo                      24267
Elemento PEP               24244
Importe libro mayor            0
Moneda libro mayor             0
Tp.cambio efectivo            76
Posición                       0
Ejercicio / mes                0
Clave contabiliz.              0
dtype: int64

In [12]:
#Agregar nombre de cuentas:
df['Nmb_Cuenta'] = df['Cuenta'].replace({
        31112701:'Leasing Equipos',
        31112702:'Evolutivos y Correctivos',
        31112703:'Redes',
        31112704:'Data Center',
        31112705:'Licencias De Software',
        31112710:'Soporte Operacional',
        31112711:'Evolutivos Mayores',
        31112712:'Soporte de Sistemas'
        })

In [15]:
#Cargar auxiliar de clasificación
nmb_aux = 'Aux_Clasificación.xlsx'
df_aux_oc = pd.read_excel(f'/content/{nmb_aux}', sheet_name='Aux_Clasificación_OC')
df_aux_txt = pd.read_excel(f'/content/{nmb_aux}', sheet_name='Aux_Clasificación_Txt')

In [36]:
#Agrupar dataframe con clasificación por OC:
df_clas = df.merge(df_aux_oc, how='left', on='Documento compras')

#Eliminar columnas duplicadas por el join:
df_clas.drop(['Cuenta_y', 'Nmb_Cuenta_y'], axis=1, inplace=True)

#Renombrar columnas modificadas por el join ("_x"):
df_clas = df_clas.rename(columns={'Cuenta_x':'Cuenta',
                                   'Nmb_Cuenta_x':'Nmb_Cuenta'})

In [ ]:
#Agrupar dataframe con clasificación por Texto:
df_clas = df.merge(df_aux_txt, how='left', on='Texto')

#Eliminar columnas duplicadas por el join:
df_clas.drop(['Cuenta_y', 'Texto_y'], axis=1, inplace=True)

#Renombrar columnas modificadas por el join ("_x"):
df_clas = df_clas.rename(columns={'Cuenta_x':'Cuenta',
                                   'Texto_x':'Texto'})

In [44]:
print(df.shape)
print(df_clas.shape)

(24267, 27)
(24267, 29)


In [45]:
df_clas.columns

Index(['Período contable', 'Texto cab.documento', 'Texto', 'Cuenta',
       'Importe en moneda local', 'Moneda local', 'Centro de coste',
       'Referencia', 'Material', 'Nombre del usuario', 'Nº documento',
       'Fecha de documento', 'Fe.contabilización', 'Importe en ML3',
       'Clase de documento', 'Orden', 'Sociedad', 'Documento compras', 'Grafo',
       'Elemento PEP', 'Importe libro mayor', 'Moneda libro mayor',
       'Tp.cambio efectivo', 'Posición', 'Ejercicio / mes',
       'Clave contabiliz.', 'Nmb_Cuenta', 'Clasificación_N1',
       'Clasificación_N2'],
      dtype='object')

#**Exportar archivo**#

In [13]:
#Nombre del archivo de salida:
nmb_output = nmb_input.replace('Cuentas TI','Gastos TI')

In [ ]:
#Extraer archivo a excel
df.to_excel(nmb_output, index=False)